In [1]:
import ipywebgl
import numpy as np

In [2]:
w = ipywebgl.GLViewer()
w.clear_color(.8, .8, .8 ,1)
w.clear()
w.enable(depth_test=True)
w.execute_commands(execute_once=True)

In [3]:
# Multi render target setup for the scene

color_geo_buffer = w.create_framebuffer()
w.bind_framebuffer('FRAMEBUFFER', color_geo_buffer)

color_target = w.create_texture()
w.bind_texture('TEXTURE_2D', color_target)
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MAG_FILTER', 'NEAREST')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MIN_FILTER', 'NEAREST')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_S', 'CLAMP_TO_EDGE')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_T', 'CLAMP_TO_EDGE')
w.tex_storage_2d('TEXTURE_2D', 1, 'RGBA8', w.width, w.height)
w.framebuffer_texture_2d('FRAMEBUFFER', 'COLOR_ATTACHMENT0', 'TEXTURE_2D', color_target, 0)

position_target = w.create_texture()
w.bind_texture('TEXTURE_2D', position_target)
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MAG_FILTER', 'NEAREST')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MIN_FILTER', 'NEAREST')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_S', 'CLAMP_TO_EDGE')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_T', 'CLAMP_TO_EDGE')
w.tex_storage_2d('TEXTURE_2D', 1, 'RGBA16F', w.width, w.height)
w.framebuffer_texture_2d('FRAMEBUFFER', 'COLOR_ATTACHMENT1', 'TEXTURE_2D', position_target, 0)

normal_target = w.create_texture()
w.bind_texture('TEXTURE_2D', normal_target)
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MAG_FILTER', 'NEAREST')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MIN_FILTER', 'NEAREST')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_S', 'CLAMP_TO_EDGE')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_T', 'CLAMP_TO_EDGE')
w.tex_storage_2d('TEXTURE_2D', 1, 'RGBA16F', w.width, w.height)
w.framebuffer_texture_2d('FRAMEBUFFER', 'COLOR_ATTACHMENT2', 'TEXTURE_2D', normal_target, 0)

depth_target = w.create_texture()
w.bind_texture('TEXTURE_2D', depth_target)
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MAG_FILTER', 'NEAREST')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MIN_FILTER', 'NEAREST')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_S', 'CLAMP_TO_EDGE')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_T', 'CLAMP_TO_EDGE')
w.tex_storage_2d('TEXTURE_2D', 1, 'DEPTH_COMPONENT16', w.width, w.height)
w.framebuffer_texture_2d('FRAMEBUFFER', 'DEPTH_ATTACHMENT', 'TEXTURE_2D', depth_target, 0)

w.draw_buffers(['COLOR_ATTACHMENT0', 'COLOR_ATTACHMENT1', 'COLOR_ATTACHMENT2'])


# ssao render target

occlusion_buffer = w.create_framebuffer()
w.bind_framebuffer('FRAMEBUFFER', occlusion_buffer)

occlusion_target = w.create_texture()
w.bind_texture('TEXTURE_2D', occlusion_target)
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MAG_FILTER', 'NEAREST')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MIN_FILTER', 'NEAREST')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_S', 'CLAMP_TO_EDGE')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_T', 'CLAMP_TO_EDGE')
w.tex_storage_2d('TEXTURE_2D', 1, 'RGBA8', w.width, w.height)
w.framebuffer_texture_2d('FRAMEBUFFER', 'COLOR_ATTACHMENT0', 'TEXTURE_2D', occlusion_target, 0)

w.bind_framebuffer('FRAMEBUFFER', None)

w.execute_commands(execute_once=True)

In [4]:
mrt_prog = w.create_program_ext(
"""#version 300 es

//the ViewBlock that is automatically filled by ipywebgl
layout(std140) uniform ViewBlock
{
    mat4 u_cameraMatrix;          //the camera matrix in world space
    mat4 u_viewMatrix;            //the inverse of the camera matrix
    mat4 u_projectionMatrix;      //the projection matrix
    mat4 u_viewProjectionMatrix;  //the projection * view matrix
};

uniform mat4 u_world;

in vec3 in_vert;
in vec3 in_normal;

out vec4 v_color;
out vec4 v_viewposition;
out vec4 v_viewnormal;

void main() {
    vec4 pos = u_world * vec4(in_vert, 1.0);
    vec4 normal = u_world * vec4(in_normal, 0.0);

    v_color = vec4(1,1,1,1);
    v_viewposition = u_viewMatrix * pos;
    v_viewnormal = u_viewMatrix * normal;
    gl_Position = u_projectionMatrix * v_viewposition;
  }
"""
,
"""#version 300 es
precision highp float;

in vec4 v_color;
in vec4 v_viewposition;
in vec4 v_viewnormal;

layout(location=0) out vec4 color;
layout(location=1) out vec4 viewPosition;
layout(location=2) out vec4 viewNormal;

void main() {
    color = v_color;
    viewPosition = v_viewposition;
    viewNormal = vec4(normalize(v_viewnormal.xyz), 0);
}
""",
{
    'in_vert' : 0,
    'in_normal' : 1
})


In [23]:
show_texture_prog = w.create_program_ext(
"""#version 300 es

in vec2 in_vert;

void main() {
    gl_Position = vec4(in_vert, 0, 1);
}
"""
,
"""#version 300 es
precision highp float;

uniform sampler2D u_texture;

out vec4 color;

void main() {
    ivec2 size = textureSize(u_texture, 0); //so we can display smaller textures
    color = vec4(texelFetch(u_texture, ivec2(gl_FragCoord.xy) % size, 0).rgb, 1.0);
    //color = vec4(gl_FragCoord.xyz,1);
}
""",
{
    'in_vert' : 0,
})

In [6]:
sphere_vbo = w.create_buffer_ext(
    src_data=np.array(
      [[ 0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  1.  ],
       [-0.72, -0.53,  0.45, -0.72, -0.53,  0.45],
       [ 0.28, -0.85,  0.45,  0.28, -0.85,  0.45],
       [ 0.89,  0.  ,  0.45,  0.89,  0.  ,  0.45],
       [ 0.28,  0.85,  0.45,  0.28,  0.85,  0.45],
       [-0.72,  0.53,  0.45, -0.72,  0.53,  0.45],
       [-0.89, -0.  , -0.45, -0.89, -0.  , -0.45],
       [-0.28, -0.85, -0.45, -0.28, -0.85, -0.45],
       [ 0.72, -0.53, -0.45,  0.72, -0.53, -0.45],
       [ 0.72,  0.53, -0.45,  0.72,  0.53, -0.45],
       [-0.28,  0.85, -0.45, -0.28,  0.85, -0.45],
       [-0.  ,  0.  , -1.  , -0.  ,  0.  , -1.  ],
       [ 0.16, -0.5 ,  0.85,  0.16, -0.5 ,  0.85],
       [-0.43, -0.31,  0.85, -0.43, -0.31,  0.85],
       [-0.26, -0.81,  0.53, -0.26, -0.81,  0.53],
       [ 0.53, -0.  ,  0.85,  0.53, -0.  ,  0.85],
       [ 0.69, -0.5 ,  0.53,  0.69, -0.5 ,  0.53],
       [ 0.16,  0.5 ,  0.85,  0.16,  0.5 ,  0.85],
       [ 0.69,  0.5 ,  0.53,  0.69,  0.5 ,  0.53],
       [-0.43,  0.31,  0.85, -0.43,  0.31,  0.85],
       [-0.26,  0.81,  0.53, -0.26,  0.81,  0.53],
       [-0.85, -0.  ,  0.53, -0.85, -0.  ,  0.53],
       [-0.59, -0.81, -0.  , -0.59, -0.81, -0.  ],
       [-0.  , -1.  , -0.  , -0.  , -1.  , -0.  ],
       [ 0.59, -0.81,  0.  ,  0.59, -0.81,  0.  ],
       [ 0.95, -0.31, -0.  ,  0.95, -0.31, -0.  ],
       [ 0.95,  0.31, -0.  ,  0.95,  0.31, -0.  ],
       [ 0.59,  0.81, -0.  ,  0.59,  0.81, -0.  ],
       [-0.  ,  1.  , -0.  , -0.  ,  1.  , -0.  ],
       [-0.59,  0.81, -0.  , -0.59,  0.81, -0.  ],
       [-0.95,  0.31,  0.  , -0.95,  0.31,  0.  ],
       [-0.95, -0.31,  0.  , -0.95, -0.31,  0.  ],
       [-0.69, -0.5 , -0.53, -0.69, -0.5 , -0.53],
       [ 0.26, -0.81, -0.53,  0.26, -0.81, -0.53],
       [ 0.85,  0.  , -0.53,  0.85,  0.  , -0.53],
       [ 0.26,  0.81, -0.53,  0.26,  0.81, -0.53],
       [-0.69,  0.5 , -0.53, -0.69,  0.5 , -0.53],
       [-0.53, -0.  , -0.85, -0.53, -0.  , -0.85],
       [-0.16, -0.5 , -0.85, -0.16, -0.5 , -0.85],
       [ 0.43, -0.31, -0.85,  0.43, -0.31, -0.85],
       [ 0.43,  0.31, -0.85,  0.43,  0.31, -0.85],
       [-0.16,  0.5 , -0.85, -0.16,  0.5 , -0.85]], dtype=np.float32).flatten()
)
indices = np.array(
    [[0, 13, 12], [12, 14, 2], [12, 13, 14], [13, 1, 14], [0, 12, 15], [15, 16, 3], [15, 12, 16], [12, 2, 16],
          [0, 15, 17], [17, 18, 4], [17, 15, 18], [15, 3, 18], [0, 17, 19], [19, 20, 5], [19, 17, 20], [17, 4, 20],
          [0, 19, 13], [13, 21, 1], [13, 19, 21], [19, 5, 21], [1, 22, 14], [14, 23, 2], [14, 22, 23], [22, 7, 23],
          [2, 24, 16], [16, 25, 3], [16, 24, 25], [24, 8, 25], [3, 26, 18], [18, 27, 4], [18, 26, 27], [26, 9, 27],
          [4, 28, 20], [20, 29, 5], [20, 28, 29], [28, 10, 29], [5, 30, 21], [21, 31, 1], [21, 30, 31], [30, 6, 31],
          [1, 31, 22], [22, 32, 7], [22, 31, 32], [31, 6, 32], [2, 23, 24], [24, 33, 8], [24, 23, 33], [23, 7, 33],
          [3, 25, 26], [26, 34, 9], [26, 25, 34], [25, 8, 34], [4, 27, 28], [28, 35, 10], [28, 27, 35], [27, 9, 35],
          [5, 29, 30], [30, 36, 6], [30, 29, 36], [29, 10, 36], [6, 37, 32], [32, 38, 7], [32, 37, 38], [37, 11, 38],
          [7, 38, 33], [33, 39, 8], [33, 38, 39], [38, 11, 39], [8, 39, 34], [34, 40, 9], [34, 39, 40], [39, 11, 40],
          [9, 40, 35], [35, 41, 10], [35, 40, 41], [40, 11, 41], [10, 41, 36], [36, 37, 6], [36, 41, 37], [41, 11, 37]],
        dtype=np.uint8).flatten()

sphere_vao = w.create_vertex_array_ext(
    mrt_prog,
    [
        (sphere_vbo, '3f32 3f32', 'in_vert', 'in_normal'),
    ],
    indices
)

plane_vbo = w.create_buffer_ext(
    src_data=np.array(
      [[10  ,  0.  ,  -10.  ,  0.  ,  1  ,  0.  ],
       [-10  ,  0.  ,  -10.  ,  0.  ,  1  ,  0.  ],
       [-10  ,  0.  ,  10.  ,  0.  ,  1  ,  0.  ],
       [-10  ,  0.  ,  10.  ,  0.  ,  1  ,  0.  ],
       [10  ,  0.  ,  10.  ,  0.  ,  1  ,  0.  ],
       [10  ,  0.  ,  -10.  ,  0.  ,  1  ,  0.  ]], dtype=np.float32).flatten()
)

plane_vao = w.create_vertex_array_ext(
    mrt_prog,
    [
        (plane_vbo, '3f32 3f32', 'in_vert', 'in_normal'),
    ]
)

screen_vbo = w.create_buffer_ext(
    src_data=np.array(
      [-1, 1,
        -1, -1,
        1, -1,
        -1, 1,
        1, -1,
        1, 1,], dtype=np.float32).flatten()
)

screen_vao = w.create_vertex_array_ext(
    show_texture_prog,
    [
        (screen_vbo, '2f32', 'in_vert'),
    ]
)

In [7]:
# bind the different texture to the different texture unit ( to display )
w.active_texture(0)
w.bind_texture('TEXTURE_2D', color_target)
w.active_texture(1)
w.bind_texture('TEXTURE_2D', position_target)
w.active_texture(2)
w.bind_texture('TEXTURE_2D', normal_target)
w.execute_commands(execute_once=True)
    

# scene to render
spheres_count = 20
spheres = np.eye(4)[np.newaxis,...].repeat(spheres_count, axis=0)
spheres[:,:3,3] = np.random.random([spheres_count,3]) * 6 - 2.5
spheres[:,1,3] += 3
plane = np.eye(4, dtype=np.float32)

def render_scene():
    w.enable(depth_test=True)

    # render in the render targets
    w.bind_framebuffer('FRAMEBUFFER', color_geo_buffer)
    w.clear()
    w.use_program(mrt_prog)

    # draw the scene
    w.bind_vertex_array(sphere_vao)
    for i in range(spheres.shape[0]):
        w.uniform_matrix('u_world', spheres[i,:,:].T)
        w.draw_elements('TRIANGLES', indices.shape[0], 'UNSIGNED_BYTE', 0)

    w.bind_vertex_array(plane_vao)
    w.uniform_matrix('u_world', plane.T)
    w.draw_arrays('TRIANGLES', 0, 6)
    
    # release the render targets
    w.bind_framebuffer('FRAMEBUFFER', None)

In [27]:
from ipywidgets import widgets, interact

def _render(texture_id=0):
    #render the scene
    render_scene()
    
    # display the texture
    w.bind_framebuffer('FRAMEBUFFER', None)
    w.disable(depth_test=True)
    w.clear()
    w.use_program(show_texture_prog)
    w.bind_vertex_array(screen_vao)
    w.uniform('u_texture', np.array([texture_id], dtype=np.int32))
    w.draw_arrays('TRIANGLES',0, 6)

    w.execute_commands()
    
    
interact(
    _render, 
    texture_id=widgets.IntSlider(description='texture id', value=0, min=0, max=2)
)
w

interactive(children=(IntSlider(value=0, description='texture id', max=2), Output()), _dom_classes=('widget-in…

GLViewer(camera_pitch=-12.399999999999995, camera_pos=[14.330596753440854, 12.39603657634239, 18.8749421400339…

In [17]:
noise = np.zeros([4,4,4], dtype=np.uint8)
noise[:,:,:2] = np.random.random([4,4,2]) * 255 - 128

# create the noise texture to randomly rotate the kernel
# bind it to the 3rd active_texture
noise_tex = w.create_texture()
w.active_texture(3)
w.bind_texture('TEXTURE_2D', noise_tex)
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MAG_FILTER', 'NEAREST')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MIN_FILTER', 'NEAREST')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_S', 'REPEAT')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_T', 'REPEAT')
w.tex_image_2d('TEXTURE_2D', 0, 'RGBA', 4, 4, 0, 'RGBA', 'UNSIGNED_BYTE', noise.flatten())
w.execute_commands(execute_once=True)

In [26]:
# display the texture
w.bind_framebuffer('FRAMEBUFFER', None)
w.disable(depth_test=True)
w.clear()
w.use_program(show_texture_prog)
w.bind_vertex_array(screen_vao)
w.uniform('u_texture', np.array([3], dtype=np.int32))
w.draw_arrays('TRIANGLES',0, 6)

w.execute_commands()
w._resources[-2]

GLResourceWidget(uid=27)

In [31]:
ssao_prog = w.create_program_ext(
"""#version 300 es

in vec2 in_vert;

void main() {
    gl_Position = vec4(in_vert, 0, 1);
}
"""
,
"""#version 300 es
precision highp float;

uniform vec3 u_kernels[16];

out vec4 color;

void main() {
    
    color = vec4(u_kernels[1], 1.0);
}
""",
{
    'in_vert' : 0,
})

In [52]:
kernels = np.random.random([16,3]).astype(np.float32)
kernels *= np.array([2,2,1], dtype=np.float32)
kernels -= np.array([1,1,0], dtype=np.float32)
kernels /= np.linalg.norm(kernels, axis=1).reshape([16,1])
kernels *= (np.random.random([16,1]) *.9 + .1)
kernels

array([[ 0.07171199, -0.2393142 ,  0.1365647 ],
       [-0.0785199 , -0.66233546,  0.49068156],
       [ 0.1103247 ,  0.16068085,  0.19044289],
       [-0.61952263,  0.10307277,  0.57451403],
       [-0.45129183,  0.03998354,  0.8670323 ],
       [ 0.10606628,  0.1081762 ,  0.1044715 ],
       [-0.05906016,  0.29033434,  0.39874473],
       [-0.31299192,  0.35514635,  0.5923701 ],
       [ 0.02253142, -0.2952357 ,  0.25511855],
       [ 0.16188839, -0.18868977,  0.18863136],
       [ 0.25384015,  0.02816768,  0.21439104],
       [ 0.11896226, -0.1460125 ,  0.15539618],
       [ 0.6282319 ,  0.08679377,  0.39025736],
       [-0.5042479 ,  0.12021536,  0.3458164 ],
       [-0.18128932,  0.0046703 ,  0.21767282],
       [-0.62848794,  0.15743579,  0.3832499 ]], dtype=float32)

In [53]:
w.clear()
w.use_program(ssao_prog)
w.uniform('u_kernels[0]', kernels)
w.use_program(show_texture_prog)
w.bind_vertex_array(screen_vao)
w.uniform('u_texture', np.array([3], dtype=np.int32))
w.draw_arrays('TRIANGLES',0, 6)
w.execute_commands()